In [30]:
import os
import torchaudio
from torch.utils.data import Dataset
import torch
import pandas as pd
import shutil
import librosa

### Recursively get every audio file

In [2]:
def audio_fetch(src,audio_paths):
    l=os.listdir(src)
    if(len(l)!=0):
        for i in range(len(l)):
            if(".wav" in l[i] or ".mp3" in l[i] or ".aac" in l[i]):
                #print(str(src+l[i]))
                audio_paths.append(str(src+l[i]))
            elif("." not in l[i]):
                try:
                    audio_fetch(str(src+"/"+l[i]+"/"),audio_paths)
                except:
                    continue
                            
def class_fetch(audio_paths)->(list,dict):
    s=set()
    for i in audio_paths:
        s.add(i.split("/")[-2])
    d={}
    count=0
    for i in s:
        d[i]=count
        count+=1
    return list(s),d


In [3]:
audio_files=[]
audio_fetch("Sound_Classification/archive/respiratory_sound_database",audio_files)
to_combine=dict()

In [4]:
for i in range(101,227):
    to_combine[i]=[]

In [5]:
audio_files.sort()

In [6]:
for i in range(len(audio_files)):
    to_combine[int(audio_files[i].split("/")[-1].split("_")[0])].append(audio_files[i])

In [7]:
to_combine[101].append("moose")

In [8]:
audio_files

['Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/101_1b1_Al_sc_Meditron.wav',
 'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/101_1b1_Pr_sc_Meditron.wav',
 'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/102_1b1_Ar_sc_Meditron.wav',
 'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/103_2b2_Ar_mc_LittC2SE.wav',
 'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/104_1b1_Al_sc_Litt3200.wav',
 'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/104_1b1_Ar_sc_Litt3200.wav',
 'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/104_1b1_Ll_sc_Litt3200.wav',
 'Sound_Classification/archive/respiratory_sound_databa

In [9]:
to_combine

{101: ['Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/101_1b1_Al_sc_Meditron.wav',
  'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/101_1b1_Pr_sc_Meditron.wav',
  'moose'],
 102: ['Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/102_1b1_Ar_sc_Meditron.wav'],
 103: ['Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/103_2b2_Ar_mc_LittC2SE.wav'],
 104: ['Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/104_1b1_Al_sc_Litt3200.wav',
  'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/104_1b1_Ar_sc_Litt3200.wav',
  'Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database//audio_and_txt_files/104_1b1_Ll_sc_Litt3200.wav',
  'Sound_Class

In [10]:
len(audio_files)

920

In [11]:
df=pd.read_csv("Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database/patient_diagnosis.csv",header=None)

In [12]:
df[1].value_counts()

1
COPD              64
Healthy           26
URTI              14
Bronchiectasis     7
Pneumonia          6
Bronchiolitis      6
LRTI               2
Asthma             1
Name: count, dtype: int64

In [13]:
def audio_fetch(src,audio_paths):
    l=os.listdir(src)
    if(len(l)!=0):
        for i in range(len(l)):
            if(".wav" in l[i] or ".mp3" in l[i] or ".aac" in l[i]):
                audio_paths.append(str(src+l[i]))
            elif("." not in l[i]):
                try:
                    audio_fetch(str(src+"/"+l[i]+"/"),audio_paths)
                except:
                    continue
                            
def class_fetch(df: pd.DataFrame())->(list,dict):
    s=set(sorted(df[1].unique()))
    toDel=[i for i in dict(df[1].value_counts(sort=True)).keys() if dict(df[1].value_counts(sort=True))[i]<3]
    for i in toDel:
        s.remove(i)
    d={}
    
    toDel_ids=[]
    for i in toDel:
        toDel_ids.extend(list(df[df[1]==i][0].values))
    
    count=0
    for i in s:
        d[i]=count
        count+=1
    return list(s),d, toDel_ids

class audio_dataset(Dataset):
    def __init__(self,audio_path: str,info: pd.DataFrame(),header=None) -> None:
        self.paths=[]
        audio_fetch(audio_path,self.paths)
        self.patient=dict()
        self.paths.sort()
        
        for i in range(101,227):
            self.patient[i]=[]
        
        for i in range(len(self.paths)):
            self.patient[int(self.paths[i].split("/")[-1].split("_")[0])].append(self.paths[i])
        
        self.info_df=pd.read_csv(info,header=header)
        self.classes, self.class_to_idx, self.ids_to_remove = class_fetch(self.info_df)

        for i in self.ids_to_remove:
            self.patient.pop(i)


        #Denotes sequence of audio files for a particular patient
        self.sequence=list(self.patient.values())


    def get_class(self,patient: list):
        return self.info_df[self.info_df[0]==int(patient[0].split("/")[-1].split("_")[0])][1].values[0]

        
    def __len__(self) -> int:
        return len(self.sequence)

    #def combine_audio() -> torch.Tensor():

    def __getitem__(self, index: int) -> torch.Tensor():
        self.val=self.sequence[index]
        return self.val,self.get_class(self.val)

In [14]:
ds=audio_dataset("Sound_Classification/archive/respiratory_sound_database","Sound_Classification/archive/respiratory_sound_database/Respiratory_Sound_Database/patient_diagnosis.csv")

In [15]:
ds.get_class(ds.sequence[6])

'COPD'

In [16]:
df=torch.utils.data.DataLoader(ds,batch_size=32)

In [28]:
wave,samplerate=torchaudio.load(ds.sequence[0][0])
#resampler= torchaudio.transforms.Resample(samplerate,40000)
#resampler(wave).shape
wave.shape

torch.Size([1, 882000])

In [18]:
wave.shape

torch.Size([1, 882000])

In [19]:
samplerate

44100

In [20]:
t1=torch.tensor([1,2,3])
t2=torch.tensor([4,5])

In [21]:
torch.concat((t1,t2))

tensor([1, 2, 3, 4, 5])

In [25]:
y, s = librosa.load(ds.sequence[0][0], sr=40000)

In [27]:
y.shape

(800000,)

In [52]:
t1=torch.tensor([])

In [53]:
for i in ds.sequence[0]:
    wv, sr=torchaudio.load(i)
    resample=torchaudio.transforms.Resample(sr,40000)
    print(resample(wv)[0].shape)
    t1=torch.concat((t1,resample(wv)[0]))
    #print(t1)

torch.Size([800000])
torch.Size([800000])


In [55]:
t1.shape

torch.Size([1600000])

In [57]:
torchaudio.save("./wyay.wav",torchaudio.load(ds.sequence[0][0])[0],44000)